In [65]:
# Import Packages 

import numpy as np
import scipy.interpolate
import scipy.optimize as opt
import copy as copy
import matplotlib.pyplot as plt
from scipy.interpolate import PchipInterpolator as pchip

In [66]:
# Define Parameters

beta = 0.8
R = 1.02
alpha = 1/3

In [67]:
# Define Primitive Functions 

def u(c):
    if c >= 0.001:
        return np.log(c)
    else:
        return - 10000

def u_dash(c):
    return 1/c

def f(k):
    return k**alpha

def f_dash(k):
    return alpha*k**(alpha-1)


In [68]:
# Calculate optimal k 

def calculate_optimal_k(R, alpha): 
    return (R / alpha) ** (1 / (alpha - 1))

k_star = calculate_optimal_k(R, alpha)

In [69]:
# Functions to solve FOCs 

def threshold_FOC(w, phi, vprime):
    target = u_dash(w - (1-phi) * k_star) - beta * R * vprime((k_star **alpha) - R * phi * k_star) 
    return target 

def bonds_foc(b, w, phi, v_prime): ## FOC for unconstrained agent
    target_0 = u_dash(w - b - k_star) - beta * R * v_prime(f_dash(k_star) + R*b) 
    return target_0

def capital_foc(k, w, phi, v_dash):
    target = u_dash(w - (1 - phi) * k) * (1 - phi) - beta * v_dash(f(k) - R * phi * k) * (f_dash(k) - R * phi)
    return target

def solve_for_bonds(w, phi, v_dash, initial_guess):
    b = opt.fsolve(bonds_foc, initial_guess, args = (w, phi, v_dash))
    return b

def solve_for_capital(w, phi, v_dash, initial_guess): 
    k = opt.fsolve(capital_foc, initial_guess, args = (w, phi, v_dash))
    return k


In [89]:
# Set up grid

w_grid = np.linspace(0.1, 5, 100)

# Define initial guess for threshold value of w 

w_threshold = 0.2


In [92]:
# Process of value function iteration 

def vfi(phi): 

    k_star = calculate_optimal_k(R, alpha)

    # Initialize policy functions 

    b_policy = np.zeros(len(w_grid))
    k_policy = np.ones(len(w_grid))

    # Initialize value function

    v = np.log(w_grid)

    # Form a loop to iterate until we are below a specified tolerance

    tol = 1e-6

    error = 1

    iter = 0

    while error > tol and iter < 10:

        iter = iter + 1

        # Create a smooth differentiable approximation to the value function

        v_approx = pchip(w_grid, v, )

        # plot v_approx and v against w_grid 

        # plt.plot(w_grid, v_approx(w_grid), label = 'v_approx')
        # plt.plot(w_grid, v, label = 'v')
        # plt.legend()
        # plt.show()

        # Calculate derivative of v_approx

        v_approx_dash = v_approx.derivative(1)

        # Plot derivative of v_approx

        # plt.plot(w_grid, v_approx_dash(w_grid))
        # plt.show()

        # Given v_approx_dash, calculate threshold value of w: 

        w_threshold = opt.fsolve(threshold_FOC, 1, args = (phi, v_approx_dash))

        k_guess = 0.01 
        b_guess = 0.01

        v_new = np.zeros(len(w_grid))

        for i in range(len(w_grid)):

            # Calculate policy functions for w < w_threshold, which corresponds to the constrained agent

            if w_grid[i] < w_threshold: 

                k_policy[i] = solve_for_capital(w_grid[i], phi, v_approx_dash, k_guess)
                b_policy[i] = - phi * k_policy[i]

                v_new[i] = u(w_grid[i] - (1 - phi) * k_policy[i]) + beta * v_approx(f(k_policy[i]) - R * phi * k_policy[i])

            else: 

                # Calculate policy functions for w >= w_threshold, which corresponds to the unconstrained agent

                b_policy[i] = solve_for_bonds(w_grid[i], phi, v_approx_dash, b_guess)
                k_policy[i] = k_star

                v_new[i] = u(w_grid[i] - b_policy[i] - k_star) + beta * v_approx(f(k_star) + R * b_policy[i])

            # Update guess for k and b

            k_guess = k_policy[i]
            b_guess = b_policy[i]

        error = np.max(np.abs(v_new - v))

        print(error)

        v = copy.deepcopy(v_new)

        ## plot v_new agaist w_grid

        # plt.plot(w_grid, v_new)
        # plt.show()

        # print(v)
        print("Capital Policy")
        print(k_policy)
        print("Bonds Policy")
        print(b_policy)

    return v, b_policy, k_policy, w_threshold
        

In [94]:
v, b_policy, k_policy, w_thrshold = vfi(0.8)

1.0546653868238378
Capital Policy
[0.07490865 0.09939406 0.11888277 0.13447921 0.14719809 0.15774602
 0.16660538 0.17413279 0.18059553 0.18619661 0.18681763 0.18681763
 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763
 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763
 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763
 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763
 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763
 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763
 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763
 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763
 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763
 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763
 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763
 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763 0.18681763
 0.18681763 0.18681763 0.186